In [7]:
#libraries
import pandas as pd
import geopy
from geopy import geocoders
from geopy.geocoders import Nominatim
import folium
from folium import plugins

#Holoviz time
import hvplot
import hvplot.pandas
import panel as pn
import holoviews.plotting.bokeh
import holoviews as hv
from holoviews import opts
import param

#Image export
import io
from PIL import Image
import selenium
import imgkit

In [2]:
#Initialize with username
gn = geocoders.GeoNames(username = 'BenjaminBoughton')
geolocator = Nominatim(user_agent = 'Nav_app')



In [3]:
#General helper functions
def build_locations_df(user_locations):
    # First there will be a prompt for locations - so these will be known ahead of time before constructing df



    locations_df = pd.DataFrame(columns = ['Region', 'Name', 'Longitude', 'Latitude'], 
                            index = [i for i in range(0, len(user_locations))])



    #Populate df with geocode
    for i in range(0, len(locations_df)):

        current_location = geolocator.geocode(user_locations[i], language="en")
    
        region = current_location.address.split(',')[-1].lstrip()
        name = current_location.address
        longitude = current_location.longitude
        latitude = current_location.latitude
    
    
        locations_df.loc[i, 'Region'] = region
        locations_df.loc[i, 'Name'] = name
        locations_df.loc[i, 'Longitude'] = longitude
        locations_df.loc[i, 'Latitude'] = latitude

    return locations_df

def build_map(locations_df):


    locations_df.reset_index(drop=False, inplace=True)
    locations_df.rename(columns={'index': 'icon_num'}, inplace=True)
    locations_df['icon_num'] = locations_df['icon_num'] + 1
    long_mean = locations_df['Longitude'].mean()
    lat_mean = locations_df['Latitude'].mean()

    #Need list of lists of long lats
    longitudes = [long for long in locations_df['Longitude']]
    latitudes = [lat for lat in locations_df['Latitude']]

    long_lats = [[latitudes[i], longitudes[i]] for i in range(0, len(longitudes))]
    
    map_plot_route = folium.Map(location = [lat_mean, long_mean], zoom_start = 5, tiles='CartoDB positron')

    #Markers
    locations_df.apply(lambda row: folium.Marker(location = [row['Latitude'], row['Longitude']],
                                         popup=row['Name'],
                                         icon=plugins.BeautifyIcon(number = row['icon_num'],
                                                                  border_color='transparent',
                                                                  border_width =0.5,
                                                                  text_color=color_dict['color_red'],
                                                                  inner_icon_style = 'font-size:20px;margin:0px;')).add_to(map_plot_route), axis=1)
    
    return map_plot_route




In [4]:
#Holoviz assets and functions
from holoviz_items.holoviz_assets import *

#Global variables
user_locations = []
user_map = folium.Map(lat = 5, long = 5, zoom_start = 10)


def query_address(event):
    
    #TODO: Handle more than one result from query
    query_result = geolocator.geocode(text_input_dict['address_input'].value, exactly_one=False, language="en")
    if query_result:
        text_input_dict['address_query_display'].value = geolocator.geocode(text_input_dict['address_input'].value, exactly_one=False, language="en")[0].address
    else:
        text_input_dict['address_query_display'].value = 'None'

def add_poi(event):

    

    if text_input_dict['address_query_display'].value not in strings_dict['all_addresses_display'].object:
        if len(strings_dict['all_addresses_display'].object) > 0:
            strings_dict['all_addresses_display'].object =  strings_dict['all_addresses_display'].object + '\n' + text_input_dict['address_query_display'].value  
        else:
            strings_dict['all_addresses_display'].object = text_input_dict['address_query_display'].value
  
    globals()['user_locations'].append(text_input_dict['address_query_display'].value)
    globals()['user_locations'] = list(set(globals()['user_locations']))
    
def map_poi(event):
    component_names = [i.name for i in globals()['dashboard']]
    map_index = component_names.index([i for i in component_names if 'Folium' in i][0])
    markdown_status_index = component_names.index([i for i in component_names if 'Map Status Markdown' in i][0])
    if globals()['user_locations']:
        globals()['dashboard'][markdown_status_index].object = 'Generating map of Points of Interest...'
        user_locations_df = build_locations_df(globals()['user_locations'])
        map_plot_route = build_map(user_locations_df)
        globals()['dashboard'][map_index] = map_plot_route
        globals()['dashboard'][markdown_status_index].object = ''

        
#     query_result = geolocator.geocode(text_input_dict['address_input'].value, exactly_one=False, language="en")
#     if query_result:
#         text_input_dict['address_query_display'].value = geolocator.geocode(text_input_dict['address_input'].value, exactly_one=False, language="en")[0].address
#     else:
#         text_input_dict['address_query_display'].value = 'None'
#Link assets with functions



text_input_dict['address_input'].param.watch(query_address, 'value')
button_dict['add_address'].param.watch(add_poi, 'clicks')
button_dict['map_poi'].param.watch(map_poi, 'clicks')



Watcher(inst=Button(button_type='primary', name='Map Points of Interest'), cls=<class 'panel.widgets.button.Button'>, fn=<function map_poi at 0x11c6e2440>, mode='args', onlychanged=True, parameter_names=('clicks',), what='value', queued=False)

In [5]:
#Holoviz time


dashboard = pn.Column(markdown_dict['overall_header'],
                     markdown_dict['add_points'],
                     text_input_dict['address_input'],
                     pn.Row(pn.Column(text_input_dict['address_query_display'], button_dict['add_address']), pn.Column(strings_dict['current_poi'], strings_dict['all_addresses_display'])),
                     markdown_dict['line_separator'],
                     button_dict['map_poi'],
                     markdown_dict['mapping_progress'],
                     pn.panel(user_map)).servable()

In [6]:
dashboard

Column
    [0] Markdown(str, style={'font-family': '...})
    [1] Markdown(str, style={'font-family': '...})
    [2] TextInput(name='Address: ', width_policy='max')
    [3] Row
        [0] Column
            [0] TextInput(disabled=True, height=100, name='Address Found: ', width_policy='max')
            [1] Button(button_type='success', name='Add Address t...)
        [1] Column
            [0] Str(str, style={'font-family': '...})
            [1] Str(str, sizing_mode='stretch_both', style={'font-family': '...})
    [4] Markdown(str, style={'font-family': '...})
    [5] Button(button_type='primary', name='Map Points of Interest')
    [6] Markdown(str, name='Map Status Markdown', style={'font-family': '...})
    [7] Folium(Map)